# LabMaps

:::note

For small tests, we query both HAL and DBLP (default behavior), but for large retrievals we use HAL only, as it is faster.
:::

## Lab authors

Lab authors are the main ingredient to analyse a single lab (i.e. a group of researchers). You can create one just with a name and then automatically ask to retrieve the DB endpoints for this author.

In [ ]:
from gismap.lab import LabAuthor

maria = LabAuthor("Maria Potop")
maria.auto_sources()

Let's look at the sources:

In [ ]:
maria.sources

Note that an author can have many names.

In [ ]:
maria.aliases

When using `auto_source`, you can tell which DBs should be uses (only online DBLP and HAL are available right now).

In [ ]:
celine = LabAuthor("Céline Comte")
celine.auto_sources(dbs="dblp")
celine.sources

In [ ]:
celine = LabAuthor("Céline Comte")
celine.auto_sources(dbs=["hal"])
celine.sources

When the sources of an author are set one can retrieve her publications.

In [ ]:
[p for p in celine.get_publications().values() if p.year == 2021]

:::note
Lab authors can have metadata that can be used for display and further analysis. This is not covered in this tutorial.
:::

## Your first lab

In GISMAP, a LabMap is a class whose instances have three methods:

- `update_authors` automatically refresh the members of the lab. It is useful at creation or when a lab evolves.
- `update_publications` makes a full refresh of the publications of a lab. All publications from lab members are considered (temporal filtering may be enabled later).
- `expand` adds *moons*, i.e. additional researchers that gravitate around the lab.

The simplest usable subclass of Lab is `ListMap`, which uses a list of names. For example, consider the former team *Gangsta* from my Bell Labs days.

In [ ]:
from gismap.lab import ListMap

lab = ListMap(
    author_list=["Fabien Mathieu", "Philippe Jacquet", "Alonso Silva", "Anne Bouillard", "François Durand", "Amira Alloum", "Marc-Olivier Buob", "Mohamed Lamine Lamali"],
    name="Gangsta", dbs="hal"
)
lab.update_authors()
lab.update_publis()
len(lab.publications)

Maps can be saved withe the `dump` method so you don't have to re-update them all the time.

When you have a populated lab, you can produce a standalone HTML of the collaboration graph with `save_html`. That graph is a standalone HTML that can be displayed in a notebook or saved for inclusion in a web page (e.g. with `iframe`).

You can also display it directly inside a notebook:

In [ ]:
lab.show_html()

Let's add some context with a few moons.

In [ ]:
lab.expand(target=4)

In [ ]:
lab.show_html()

Few things about the generated graph:

- Authors are represented with their initials unless some picture url is provided (implicitly or explictly).
- Comets are singletons (authors with no co-publications with the other nodes). They are hidden by default. For example, if you only show the moons, Bernard becomes a comet and is hidden.
- You can hover an author to get her name. If you click, you have a modal with the list of publications.
- The width and length of an edge depend on the number of co-publications. If you click you have a modal with the list of co-publications.

## Make your own LabMap

Gismap is intended to make easy the creation of LabMaps in many contexts.

The easiest way to manage a lab, apart from using ListMaps as shown above, is to specify an internal method `_author_iterator` that returns Lab authors. When it's done, you can create/refresh LabMaps as you see fit.

How the iterator works is 100% up to you. Most of the time, this is done by scrapping some Web page(s) (see the gallery for examples), but many other options exist, e.g. read authors from a file, from a LDAP...